In [ ]:
# File: fitting.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import os
import random
import tempfile

from frontend import App

# create an app
app = App.create("fitting")

# set up paths to Codim-IPC repository assets
codim_ipc_root = os.path.join(tempfile.gettempdir(), "Codim-IPC")
checkout_list = [
    "Projects/FEMShell/input/dress_knife",
    "Projects/FEMShell/input/Rumba_Dancing",
]

# sparse clone only needed directories from the repository
app.extra.sparse_clone(
    "https://github.com/ipc-sim/Codim-IPC", codim_ipc_root, checkout_list
)

stage_path = os.path.join(codim_ipc_root, checkout_list[0], "stage.obj")
body_path = os.path.join(codim_ipc_root, checkout_list[1], "shell0.obj")

# load dress mesh with stitching information
V, F, S = app.extra.load_CIPC_stitch_mesh(stage_path)
app.asset.add.tri("dress", V, F)
app.asset.add.stitch("glue", S)

# load body mesh
V, F = app.mesh.load_tri(body_path)
app.asset.add.tri("body", V, F)

# create a scene
scene = app.scene.create()

# add dress with stitching and rotation
dress = scene.add("dress")
dress.param.set("friction", 0.0).set("strain-limit", 0.1)
dress.stitch("glue").rotate(-90, "x")

# add body with animated motion from sequence of meshes
jitter = 0.01 * random.random()
body = scene.add("body").at(0, -0.78 + jitter, 0)
body.param.set("friction", 0.0).set("model", "arap")
pin = body.pin()

# load body animation sequence from files
body_dir = os.path.join(codim_ipc_root, checkout_list[1])
frame = 1
while True:
    path = os.path.join(body_dir, f"shell{frame}.obj")
    if os.path.exists(path):
        V, _ = app.mesh.load_tri(path)
        t0, t1 = 0.5 + 0.025 * (frame - 1), 0.5 + 0.025 * frame
        pin.move_to(V, t0, t1)
        frame += 1
    else:
        break

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview(options={"pin": False})

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters with dynamic fitting mode
param = session.param.set("fitting").set("dt", 1e-3).set("frames", 240)

# configure dynamic parameters to transition out of fitting mode
param.dyn("fitting").time(0.15).hold().change(False)
param.dyn("dt").time(0.15).hold().change(0.01)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate()

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()